Cleaning Bank Marketing Data
===========================
_Instructions and dataset taken from Datacamp's [Cleaning Bank Marketing Campaign Data](https://app.datacamp.com/learn/projects/1613)_


# Objectives
- apply data-cleaning skills to process information about marketing campaigns run by a bank
-modify values, add new features, convert data types as required by the client
- save data into multiple files

_The contents of this cell are taken directly from Datacamp's [Cleaning Bank Marketing Campaign Data](https://app.datacamp.com/learn/projects/1613)_

![Piggy bank](resources/piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [343]:
import pandas as pd
import numpy as np

In [344]:
df = pd.read_csv("bank_marketing.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

# Processing for _client.csv_

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

Create `clients_df` storing the required columns from the input:

In [345]:
clients_df = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]
clients_df.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin.,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


## client.csv: Check for are missing values
Let's check if there are missing values and how we can handle them, if needed.

In [346]:
print(clients_df.info(),'\n')
clients_df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       41188 non-null  object
 5   credit_default  41188 non-null  object
 6   mortgage        41188 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.2+ MB
None 



client_id         0
age               0
job               0
marital           0
education         0
credit_default    0
mortgage          0
dtype: int64

There are a total of 41,188 rows, and there appears to be no missing values for the client-related columns, so we can skip the handling of missing data.

## client.csv: client_id
As a sanity check, the `client_id` column should:
- contain 41,188 unique values
- since the id values incrementing, minimum value should be 0 while maximum should be 41187

In [347]:
# Check that unique id values are equal to the number of rows in the input df (41,188)
assert(len(clients_df['client_id'].unique()) == df.shape[0])

# Check that the minimum and maximum values are 0 and 41187, respectively
assert(clients_df['client_id'].min() == 0)
assert(clients_df['client_id'].max() == (df.shape[0] - 1))

## client.csv: age
As a sanity check, the `age` column should:
- be positive integers greater than 0

Checking below, the youngest client is 17 years old while the oldest is 98. No further processing is required for `age` .

In [348]:
print(clients_df['age'].min(), clients_df['age'].max())

assert(clients_df['age'].min() > 0)
assert(clients_df['age'].max() > 0)

17 98


## client.csv: marital
As a sanity check, the `marital` column should:
- be uniform in terms of case
- not have trailing or leading whitespaces

Checking below, there are four (4) unique values in the column.
Although the 'unknown' value is somewhat alarming, we will just leave it as-is.

No further processing is required for `marital` .

In [349]:
print(f"unique values for marital: {clients_df['marital'].unique()}.")

unique values for marital: ['married' 'single' 'divorced' 'unknown'].


## client.csv: `job`
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |

Let's take a peek at what kind of values have an offending `.` character:

In [350]:
# Get rows with 'job' containing a dot "." character
# Note the double-escape character needed for str.contains()
clients_job_withdot = clients_df['job'].str.contains('\\.')
print(f"number of job values with '.': {clients_job_withdot.sum()}")
clients_df[ clients_job_withdot ]['job'].unique()

number of job values with '.': 10422


array(['admin.'], dtype=object)

We found 10,422 entries with a `job` value of "admin.". Let's do a simple replace for the offending rows.

Afterwards, we can do a quick check afterwards that there are no more offending rows.

In [351]:
# Replace "."s with "_"s
# Note that escape character is not neeed for str.replace()
clients_df.loc[ clients_job_withdot, 'job'] = clients_df[ clients_job_withdot ]['job'].str.replace(".", "_")

# check that there are no more "job" values with "." character
clients_job_withdot_afterclean = clients_df['job'].str.contains('\\.')
assert(clients_job_withdot_afterclean.sum() == 0)

## client.csv: education
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |

Taking a peek at the offending rows with "." characters:

In [352]:
clients_educ_withdot = clients_df['education'].str.contains('\\.')
print(f"number of education values with '.': {clients_educ_withdot.sum()}")
clients_df[ clients_educ_withdot ]['education'].unique()

number of education values with '.': 39439


array(['basic.4y', 'high.school', 'basic.6y', 'basic.9y',
       'professional.course', 'university.degree'], dtype=object)

We found 39,439 entries with a `education` value with the "." character. Let's do a simple replace for the offending rows.

Afterwards, we can do a quick check afterwards that there are no more offending rows.

In [353]:
clients_df.loc[ clients_educ_withdot, 'education'] = clients_df[ clients_educ_withdot ]['education'].str.replace(".", "_")

# check that there are no more "education" values with "." character
clients_educ_withdot_afterclean = clients_df['education'].str.contains('\\.')
assert(clients_educ_withdot_afterclean.sum() == 0)

Next, let's change the "unknown" values to `np.NaN`.
Let's quickly view first the unique values in the column and check the count of offending rows.

In [354]:
print(f"unique education values: {clients_df['education'].unique()}\n")

client_educ_unknown = clients_df['education'] == "unknown"
num_client_educ_unknown = client_educ_unknown.sum()
print(f"number of \"unknown\" education values: {num_client_educ_unknown}")


unique education values: ['basic_4y' 'high_school' 'basic_6y' 'basic_9y' 'professional_course'
 'unknown' 'university_degree' 'illiterate']

number of "unknown" education values: 1731


There are 1,731 rows to change. Let's proceed with the replacement and assert that there are no more offending rows.

Then we're all done with the `education` column.

In [ ]:
clients_df.loc[client_educ_unknown, 'education'] = clients_df[ client_educ_unknown ]['education'].replace(to_replace="unknown", value=np.nan)

# check that there are no more "unknown" "education" values,
# and that there is the same number of null values as there were previously unknown values
client_educ_unknown_afterclean = clients_df['education'] == "unknown"
assert(client_educ_unknown_afterclean.sum() == 0)
assert( clients_df['education'].isna().sum() == num_client_educ_unknown )

C:\Users\jfmri\AppData\Local\Temp\ipykernel_8812\3473244188.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  clients_df.loc[ client_educ_unknown, 'education'] = clients_df[ client_educ_unknown ]['education'].replace(to_replace="unknown", value=np.nan)


NOTE: The `FutureWarning` appearning above is appearing due to replacing a strint type (i.e. `"unknown"`) with a float type (i.e. `np.nan`). This could solved if the whole column was initialized as a float type. That, however, is not an option here due to the `education` column containing string type data.

Since the requirements specifically call for `np.nan`, we can opt to ignore this `FutureWarning`.

## client.csv: credit_default
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
 
Let's take a peek at the unique `credit_default` values and the number of entries with each value:

In [356]:
client_creditdefault_counts_df = clients_df['credit_default'].value_counts()
client_creditdefault_counts_df

credit_default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64

There are three (3) unique values ("yes", "no", "unknown") with their counts shown and stored above.

Let's do the replacement as follows:
* "yes" -> 1
* any other values -> 0

Afterwards do some checks on the counts. Then we're done with the `credit_default` column

In [357]:
# Rename the credit_default column to credit_default_obj to reflect its datatype
clients_df = clients_df.rename(columns={"credit_default":"credit_default_obj"})

# Create a new credit_default column, set its values to 1 or 0 depending on the old credit_default
clients_df['credit_default'] = (clients_df['credit_default_obj'] == "yes")

# Drop the old credit_default_obj column
clients_df = clients_df.drop(columns=['credit_default_obj'])

# Check that number of 'True' 'credit_default' rows is equal to the number of rows previously "yes",
# and that the number of 'False' rows is equal to the sum of "no"s and "unknowns" previously
assert((clients_df['credit_default'] == True).sum() == client_creditdefault_counts_df["yes"])
assert((clients_df['credit_default'] == False).sum() ==
     (client_creditdefault_counts_df["no"] + client_creditdefault_counts_df["unknown"]))

## client.csv: mortgage
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

In [358]:
client_mortgage_counts_df = clients_df['mortgage'].value_counts()
client_mortgage_counts_df

mortgage
yes        21576
no         18622
unknown      990
Name: count, dtype: int64

The `mortgage` column is in the same situation with the `credit_default` in the previous section, so let's apply the same process.

Then we're done with the 'mortgage' column.

In [359]:
# Rename the mortgage column to mortgage_obj to reflect its datatype
clients_df = clients_df.rename(columns={"mortgage":"mortgage_obj"})

# Create a new credit_mortgage column, set its values to 1 or 0 depending on the old mortgage_obj
clients_df['mortgage'] = (clients_df['mortgage_obj'] == "yes")

# Drop the old credit_default_obj column
clients_df = clients_df.drop(columns=['mortgage_obj'])

# Check that number of 'True' 'credit_default' rows is equal to the number of rows previously "yes",
# and that the number of 'False' rows is equal to the sum of "no"s and "unknowns" previously
assert((clients_df['mortgage'] == True).sum() == client_mortgage_counts_df["yes"])
assert((clients_df['mortgage'] == False).sum() ==
     (client_mortgage_counts_df["no"] + client_mortgage_counts_df["unknown"]))

## client.csv: Export to csv
At this point, all our clean client data is stored in `clients_df`. Let's do some quick final visual checks and then export it to a CSV file.

In [360]:
clients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  bool  
 6   mortgage        41188 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 1.6+ MB


In [361]:
clients_df.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False


In [362]:
PATH_TO_CLIENTS_CSV_FILE = 'output/client.csv'
clients_df.to_csv(PATH_TO_CLIENTS_CSV_FILE, index=False)

In [363]:
# Check first few rows of exported CSV file
with open(PATH_TO_CLIENTS_CSV_FILE) as csvfile:
    for i in range(0,12):
        print(csvfile.readline())

client_id,age,job,marital,education,credit_default,mortgage

0,56,housemaid,married,basic_4y,False,False

1,57,services,married,high_school,False,False

2,37,services,married,high_school,False,True

3,40,admin_,married,basic_6y,False,False

4,56,services,married,high_school,False,False

5,45,services,married,basic_9y,False,False

6,59,admin_,married,professional_course,False,False

7,41,blue-collar,married,,False,False

8,24,technician,single,professional_course,False,True

9,25,services,single,high_school,False,True

10,41,blue-collar,married,,False,False



At this point, `client.csv` is ready.

# Processing for campaign.csv

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

Quickly view the info about the given `df` then get the required columns from `df` and store into **`campaign_df`**
Note that the required column `last_contact_date` is a new column depending on `month` and `day`, so we store those two(2) columns instead for now.

In [364]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [365]:
campaign_df = df[['client_id', 'number_contacts', 'contact_duration',
                'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'month', 'day']]
campaign_df.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day
0,0,1,261,0,nonexistent,no,may,13
1,1,1,149,0,nonexistent,no,may,19
2,2,1,226,0,nonexistent,no,may,23
3,3,1,151,0,nonexistent,no,may,27
4,4,1,307,0,nonexistent,no,may,3


## campaign.csv: Check for missing values
Let's check if there are missing values and how we can handle them, if needed.

In [366]:
print(campaign_df.info())
campaign_df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  int64 
 1   number_contacts             41188 non-null  int64 
 2   contact_duration            41188 non-null  int64 
 3   previous_campaign_contacts  41188 non-null  int64 
 4   previous_outcome            41188 non-null  object
 5   campaign_outcome            41188 non-null  object
 6   month                       41188 non-null  object
 7   day                         41188 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 2.5+ MB
None


client_id                     0
number_contacts               0
contact_duration              0
previous_campaign_contacts    0
previous_outcome              0
campaign_outcome              0
month                         0
day                           0
dtype: int64

There are a total of 41,188 rows, and there appears to be no missing values for the campaign-related columns, so we can skip the handling of missing data.

## campaign.csv: client_id
As a sanity check, the `client_id` column should:
- contain 41,888 unique values
- if incrementing, minimum value should be 0 while maximum should be 41187

Checking below, no further processing is required for `client_id` .

In [367]:
# Check that unique id values are equal to the number of rows in the input df (41,188)
assert(len(campaign_df['client_id'].unique()) == df.shape[0])

# Check that the minimum and maximum values are 0 and 41187, respectively
assert(campaign_df['client_id'].min() == 0)
assert(campaign_df['client_id'].max() == (df.shape[0] - 1))

## campaign.csv: number_contacts
As a sanity check, the `number_contacts` column should:
- be positive integers greater than 0

No further processing is required for `number_contacts` .

In [368]:
print(campaign_df['number_contacts'].min(), campaign_df['number_contacts'].max())

assert(campaign_df['number_contacts'].min() >= 0 )
assert(campaign_df['number_contacts'].max() >= 0 )

1 56


## campaign.csv: contact_duration
As a sanity check, the `contact_duration` column should:
- be positive integers greater than 0

No further processing is required for `contact_duration` .

In [369]:
print(campaign_df['contact_duration'].min(), campaign_df['contact_duration'].max())

assert(campaign_df['contact_duration'].min() >= 0 )
assert(campaign_df['contact_duration'].max() >= 0 )

0 4918


## campaign.csv: previous_campaign_contacts
As a sanity check, the `previous_campaign_contacts` column should:
- be positive integers greater than 0

No further processing is required for `previous_campaign_contacts` .

In [370]:
print(campaign_df['previous_campaign_contacts'].min(), campaign_df['previous_campaign_contacts'].max())

assert(campaign_df['previous_campaign_contacts'].min() >= 0 )
assert(campaign_df['previous_campaign_contacts'].max() >= 0 )

0 7


## campaign.csv: previous_outcome
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |

In [371]:
campaign_df.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day
0,0,1,261,0,nonexistent,no,may,13
1,1,1,149,0,nonexistent,no,may,19
2,2,1,226,0,nonexistent,no,may,23
3,3,1,151,0,nonexistent,no,may,27
4,4,1,307,0,nonexistent,no,may,3


In [372]:
campaign_prevoutcome_counts_df = campaign_df['previous_outcome'].value_counts()
campaign_prevoutcome_counts_df

previous_outcome
nonexistent    35563
failure         4252
success         1373
Name: count, dtype: int64

There are three (3) unique values ("success", "failure", "nonexistent") with their counts shown and stored above.

Let's do the replacement as follows:
* "success" -> 1
* any other value -> 0

Afterwards do some checks on the counts. Then we're done with the 'previous_outcome' column

In [373]:
# Rename the previous_outcome column to previous_outcome_obj to reflect its datatype
campaign_df = campaign_df.rename(columns={"previous_outcome":"previous_outcome_obj"})

# Create a new previous_outcome column, set its values to 1 or 0 depending on the old previous_outcome_obj
campaign_df['previous_outcome'] = (campaign_df['previous_outcome_obj'] == "success")

# Drop the old previous_outcome_obj column
campaign_df = campaign_df.drop(columns=['previous_outcome_obj'])

# Check that number of 'True' 'previous_outcome' rows is equal to the number of rows previously "success",
# and that the number of 'False' rows is equal to the sum of "nonexistent"s and "failure"s previously
assert((campaign_df['previous_outcome'] == True).sum() == campaign_prevoutcome_counts_df["success"])
assert((campaign_df['previous_outcome'] == False).sum() ==
     (campaign_prevoutcome_counts_df["nonexistent"] + campaign_prevoutcome_counts_df["failure"]))

## campaign.csv: campaign_outcome
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |

In [374]:
campaign_campoutcome_counts_df = campaign_df['campaign_outcome'].value_counts()
campaign_campoutcome_counts_df

campaign_outcome
no     36548
yes     4640
Name: count, dtype: int64

In [375]:
# Rename the campaign_outcome column to campaign_outcome_obj to reflect its datatype
campaign_df = campaign_df.rename(columns={"campaign_outcome":"campaign_outcome_obj"})

# Create a new campaign_outcome column, set its values to 1 or 0 depending on the old campaign_outcome_obj
campaign_df['campaign_outcome'] = (campaign_df['campaign_outcome_obj'] == "yes")

# Drop the old campaign_outcome_obj column
campaign_df = campaign_df.drop(columns=['campaign_outcome_obj'])

# Check that number of 'True' 'campaign_outcome' rows is equal to the number of rows previously "yes",
# and that the number of 'False' rows is equal to the sum of "no"s previously
assert((campaign_df['campaign_outcome'] == True).sum() == campaign_campoutcome_counts_df["yes"])
assert((campaign_df['campaign_outcome'] == False).sum() == campaign_campoutcome_counts_df["no"])

## campaign.csv: last_contact_date
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

Taking a quick peek first at the `month` and `day` columns below:

In [376]:
print(f"Unique months in campaign_df: {campaign_df['month'].unique()}")
print(f"Min and Max days in campaign_df: {campaign_df['day'].min()}, {campaign_df['day'].max()}")

Unique months in campaign_df: ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
Min and Max days in campaign_df: 1, 31


We see that while the `day` column is ready for conversion to `datetime` type. However, the `month` column needs to be converted from month string to the equivalent integer value prior conversion to `datetime` type.

To do that, we isolate the `month` and `day` columns into a new dataframe `last_contact_date_df`. Afterwards, do the conversion from month string to integer equivalent using

In [377]:
# Isolate the date-related columns into a new dataframe
# This is done to avoid creating unecessary new columns in the final dataframe campaign_df
last_contact_date_df = campaign_df[['day']]

# Convert month string to integer equivalent, then store to 'month' column
month_dict = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6,
              'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
last_contact_date_df = last_contact_date_df.assign(month=campaign_df['month'].map(month_dict))

# Set the year to 2022 for all entries
last_contact_date_df = last_contact_date_df.assign(year=2022)

last_contact_date_df.head()

,day,month,year
0,13,5,2022
1,19,5,2022
2,23,5,2022
3,27,5,2022
4,3,5,2022


Now that the `last_contact_date_df` is ready (i.e. day, month, and year are all in integers), let's convert them with `to_datetime` and store into the `last_contact_date` column of the final `campaign_df`.

In [378]:
# Create new column `last_contact_date` from `last_contact_date_df`
campaign_df = campaign_df.assign(last_contact_date=pd.to_datetime(arg=last_contact_date_df, format="%Y-%m-%d"))

Finally, we can drop the uneeded columns `month` and `day`, then perform some checks

In [379]:
campaign_df = campaign_df.drop(columns = ['month', 'day'])

# Check that all values in 'last_contact_date' are dates
# in between 2022-01-01(bounded) and 2023-01-01(unbounded)
assert( ((campaign_df['last_contact_date'] >= "2022-01-01") & 
         (campaign_df['last_contact_date'] < "2023-01-01")).sum() == len(campaign_df) )

# Check if month and day columns are not present anymore
assert( ("month" not in campaign_df.columns) and ("day" not in campaign_df.columns) )

campaign_df['last_contact_date'].info(), campaign_df['last_contact_date'].head(), 

<class 'pandas.core.series.Series'>
RangeIndex: 41188 entries, 0 to 41187
Series name: last_contact_date
Non-Null Count  Dtype         
--------------  -----         
41188 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 321.9 KB


(None,
 0   2022-05-13
 1   2022-05-19
 2   2022-05-23
 3   2022-05-27
 4   2022-05-03
 Name: last_contact_date, dtype: datetime64[ns])

## campaign.csv: Export to csv
At this point, all our clean campaign data is stored in `campaign_df`. Let's do some quick final visual checks and then export it to a CSV file.

In [380]:
campaign_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   last_contact_date           41188 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4)
memory usage: 1.6 MB


In [381]:
campaign_df.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,False,False,2022-05-13
1,1,1,149,0,False,False,2022-05-19
2,2,1,226,0,False,False,2022-05-23
3,3,1,151,0,False,False,2022-05-27
4,4,1,307,0,False,False,2022-05-03


In [382]:
PATH_TO_CAMPAIGN_CSV_FILE = 'output/campaign.csv'
campaign_df.to_csv(PATH_TO_CAMPAIGN_CSV_FILE, index=False)

In [383]:
# Check first few rows of exported CSV file
with open(PATH_TO_CAMPAIGN_CSV_FILE) as csvfile:
    for i in range(0,12):
        print(csvfile.readline())

client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date

0,1,261,0,False,False,2022-05-13

1,1,149,0,False,False,2022-05-19

2,1,226,0,False,False,2022-05-23

3,1,151,0,False,False,2022-05-27

4,1,307,0,False,False,2022-05-03

5,1,198,0,False,False,2022-05-05

6,1,139,0,False,False,2022-05-03

7,1,217,0,False,False,2022-05-12

8,1,380,0,False,False,2022-05-21

9,1,50,0,False,False,2022-05-05

10,1,55,0,False,False,2022-05-08



At this point, `campaign.csv` is ready.

# Processing for economics.csv

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

Quickly view the info about the given `df` then get the required columns from `df` and store into **`economics_df`**

In [384]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [385]:
economics_df = df[['client_id', 'cons_price_idx', 'euribor_three_months']]
economics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   cons_price_idx        41188 non-null  float64
 2   euribor_three_months  41188 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 965.5 KB


In [386]:
economics_df.head()

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857


## economics.csv: client_id
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |

As a sanity check, the `client_id` column should:
- contain 41,888 unique values
- if incrementing, minimum value should be 0 while maximum should be 41187

Checking below, no further processing is required for `client_id` .

In [387]:
# Check that unique id values are equal to the number of rows in the input df (41,188)
assert(len(economics_df['client_id'].unique()) == df.shape[0])

# Check that the minimum and maximum values are 0 and 41187, respectively
assert(economics_df['client_id'].min() == 0)
assert(economics_df['client_id'].max() == (df.shape[0] - 1))

## economics.csv: cons_price_idx
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |

Let's take a quick look at this column:

In [388]:
print(economics_df['cons_price_idx'].min(), economics_df['cons_price_idx'].max())
economics_df['cons_price_idx'].head()

92.201 94.767


0    93.994
1    93.994
2    93.994
3    93.994
4    93.994
Name: cons_price_idx, dtype: float64

Since there are no specific cleaning requirements and we don't have a good understanding about this value, we can leave it as-is without any further checking.

## economics.csv: euribor_three_months
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

Let's take a quick look at this column:

In [389]:
economics_df['euribor_three_months'].min(), economics_df['euribor_three_months'].max()
economics_df['euribor_three_months'].head()

0    4.857
1    4.857
2    4.857
3    4.857
4    4.857
Name: euribor_three_months, dtype: float64

As with the `cons_price_idx` column, there are no specific cleaning requirements and we don't have a good understanding about this value. We can leave it as-is without any further checking.

## economics.csv: Export to csv
At this point, all our clean campaign data is stored in `economics_df`. Let's do some quick final visual checks and then export it to a CSV file.

In [390]:
economics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   cons_price_idx        41188 non-null  float64
 2   euribor_three_months  41188 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 965.5 KB


In [391]:
economics_df.head()

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857


In [392]:
PATH_TO_ECONOMICS_CSV_FILE = 'output/economics.csv'
economics_df.to_csv(PATH_TO_ECONOMICS_CSV_FILE, index=False)

In [393]:
# Check first few rows of exported CSV file
with open(PATH_TO_ECONOMICS_CSV_FILE) as csvfile:
    for i in range(0,12):
        print(csvfile.readline())

client_id,cons_price_idx,euribor_three_months

0,93.994,4.857

1,93.994,4.857

2,93.994,4.857

3,93.994,4.857

4,93.994,4.857

5,93.994,4.857

6,93.994,4.857

7,93.994,4.857

8,93.994,4.857

9,93.994,4.857

10,93.994,4.857



At this point, `economics.csv` is ready.

# Conclusion
We have successfully extracted specific data from the given dataset `bank_marketing.csv` and performed the required data cleaning. Furthermore, we have added new columns whose values are derived from the given dataset. Finally, we were able to export them into three (3) output CSV files.

# Acknowledgements
## Course Instructor
- George Boorman
## Course Offered By
* [DataCamp](https://app.datacamp.com/)